<a href="https://colab.research.google.com/github/brandon0824/HungyiML/blob/master/HW1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import tensorflow as tf
import numpy as np

# 数据预处理
data = pd.read_csv('./drive/My Drive/Hung-yi Lee DLML HW/hw1/train.csv', encoding='big5')
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
#print(data)
#print(raw_data)

month_data = {}
for month in range(12):  # month 从0-11 共12个月
    sample = np.empty([18, 480])  # 返回一个18行480列的数组，用来保存一个月的数据（一个月只有20天，一天24个小时）
    for day in range(20):  # day从0-19 共20天
        sample[:, day * 24: (day + 1) * 24] = raw_data[18 * (20 * month + day): 18 * (20 * month + day + 1),
                                              :]  # raw的行每次取18行，列取全部列。送到sample中（sample是18行480列）行给全部行，列只给24列，然后列往后增加
    month_data[month] = sample # 按照月份进行分割
#print(month_data)



x = np.empty([12 * 471, 18 * 9],dtype = float)
y = np.empty([12 * 471, 1],dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            #一个月的最后一天的数据不能与下个月相连
            if day == 19 and hour>14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1,-1)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9,day * 24 + hour + 9]
#print(x)
#print(y)



# Normalize与训练集分类
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
#print(x)



import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
# print(x_train_set)
# print(y_train_set)
# print(x_validation)
# print(y_validation)
# print(len(x_train_set))
# print(len(y_train_set))
# print(len(x_validation))
# print(len(y_validation))

dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('./drive/My Drive/Hung-yi Lee DLML HW/hw1/weight.npy', w)
# print(w)




# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('./drive/My Drive/Hung-yi Lee DLML HW/hw1/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
# test_x


w = np.load('./drive/My Drive/Hung-yi Lee DLML HW/hw1/weight.npy')
ans_y = np.dot(test_x, w)
# ans_y



import csv
with open('./drive/My Drive/Hung-yi Lee DLML HW/hw1/submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

0:27.071214829194115
100:33.78905859777454
200:19.91375129819709
300:13.53106819368969
400:10.64546615844617
500:9.277353455475065
600:8.5180420459565
700:8.014061987588422
800:7.63675682477569
900:7.3365637403711235
['id', 'value']
['id_0', 5.174960398984736]
['id_1', 18.306214253527905]
['id_2', 20.491218094180542]
['id_3', 11.523942869805357]
['id_4', 26.61605675230614]
['id_5', 20.531348081761216]
['id_6', 21.906551018797387]
['id_7', 31.736468747068837]
['id_8', 13.391674055111729]
['id_9', 64.45646650291954]
['id_10', 20.26456883615944]
['id_11', 15.358576077361231]
['id_12', 68.58947276926726]
['id_13', 48.42811374745719]
['id_14', 18.702333824193214]
['id_15', 10.188595737466702]
['id_16', 30.74036285982045]
['id_17', 71.13221776355105]
['id_18', -4.130517391262438]
['id_19', 18.23569401642869]
['id_20', 38.578922275007756]
['id_21', 71.31151972531332]
['id_22', 7.410348162634086]
['id_23', 18.717955330321395]
['id_24', 14.937250260084578]
['id_25', 36.719736694705304]
['id_26'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
